In [5]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [7]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

prod_email = os.environ['KALSHI_PROD_EMAIL'] # change these to be your personal credentials
prod_password = os.environ['KALSHI_PROD_PASSWORD'] # (for extra security, we recommend using a config file)

demo_email = os.environ['KALSHI_DEMO_EMAIL'] # change these to be your personal credentials
demo_password = os.environ['KALSHI_DEMO_PASSWORD'] # (for extra security, we recommend using a config file)

# for prod
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

KeyError: 'KALSHI_PROD_EMAIL'

In [3]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'USDJPY-22NOV2918-T139.250', 'event_ticker': 'USDJPY-22NOV2918', 'subtitle': '>139.250', 'open_time': '2022-11-29T15:25:16Z', 'close_time': '2022-11-29T23:00:00Z', 'expiration_time': '2022-12-06T23:00:00Z', 'status': 'active', 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': 'Financials', 'risk_limit_cents': 0}

cursor: CgwIh_6NnAYQmLr0yQISFEhJR0hOWS0yMk5PVjI4LUI1My41


In [4]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'HIGHNY-22NOV28-B51.5', 'event_ticker': 'HIGHNY-22NOV28', 'subtitle': '51° to 52°', 'open_time': '2022-11-27T15:25:16Z', 'close_time': '2022-11-29T04:59:00Z', 'expiration_time': '2022-12-05T15:00:00Z', 'status': 'closed', 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': 'Climate and Weather', 'risk_limit_cents': 0}

new cursor! CgwIiLiDnAYQiNTQhwESFEhJR0hOWS0yMk5PVjI2LUI1Ny41


In [5]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'HIGHNY-22NOV27', 'series_ticker': 'HIGHNY', 'sub_title': 'On Nov 27, 2022', 'title': 'What will the high temp be in NYC on Nov 27, 2022?', 'mutually_exclusive': True, 'category': 'Climate and Weather'}

first market in event_response payload: {'ticker': 'HIGHNY-22NOV27-T57', 'event_ticker': '', 'subtitle': '56° or lower', 'open_time': '2022-11-26T15:25:15Z', 'close_time': '2022-11-28T04:59:00Z', 'expiration_time': '2022-12-04T15:00:00Z', 'status': 'closed', 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 0, 'no_ask': 0, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': '', 'risk_limit_cents': 0}


In [6]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'HIGHNY', 'frequency': 'daily', 'title': 'NYC high temp'}



In [7]:
# Next let's look at the recent market history for a market
ticker = 'NGDP-22-C7.5'

market_history_params = {'ticker': ticker,
                            'limit': 100,
                            'cursor': None,
                            'max_ts': None, # pass in unix_ts
                            'min_ts': round(time.time()-1000000) # passing a recent unix_ts
                                }
market_history_response = exchange_client.get_market_history(**market_history_params)

print('keys:', market_history_response.keys())
print()
print('most recent market history object:', market_history_response['history'][-1])
print()

# and then also look at the most current view of the orderbook
market_history_params = {'ticker':ticker,
                            'depth': 30
                                }
orderbook_response = exchange_client.get_orderbook(**market_history_params)

print('keys:', orderbook_response.keys())
print()
print('orderbook object:', orderbook_response)
print()

keys: dict_keys(['ticker', 'history', 'cursor'])

most recent market history object: {'yes_price': 23, 'yes_bid': 23, 'yes_ask': 87, 'no_bid': 13, 'no_ask': 77, 'volume': 2761, 'open_interest': 1644, 'ts': 1669651310}

keys: dict_keys(['orderbook'])

orderbook object: {'orderbook': {'yes': [[1, 515], [23, 2503]], 'no': [[1, 1146], [2, 152], [3, 38], [6, 300], [7, 38], [12, 62], [13, 154]]}}



In [8]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

{'balance': 28268}

In [9]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

{'cursor': 'CglURVNUSU5HLTU',
 'market_positions': [{'ticker': 'NGDP-22-C7.5',
   'total_traded': 1694,
   'position': -22,
   'market_exposure': 1694,
   'realized_pnl': 0,
   'resting_orders_count': 30,
   'fees_paid': 28},
  {'ticker': 'TESTING-5',
   'total_traded': 109,
   'position': 9,
   'market_exposure': 9,
   'realized_pnl': 0,
   'resting_orders_count': 39,
   'fees_paid': 1}],
 'event_positions': [{'event_ticker': 'NGDP-22',
   'total_cost': 1694,
   'event_exposure': 1694,
   'realized_pnl': 0,
   'resting_order_count': 30,
   'fees_paid': 28},
  {'event_ticker': 'TESTING-5',
   'total_cost': 109,
   'event_exposure': 9,
   'realized_pnl': 0,
   'resting_order_count': 39,
   'fees_paid': 1}]}

In [10]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {'ticker':None,
                    'order_id':None,
                    'min_ts':None,
                    'max_ts':None,
                    'limit': None,
                    'cursor': None}

fills = exchange_client.get_fills(**fills_params)
fills

{'fills': [{'trade_id': '59f55de3-0a69-47fe-82ce-298ba48c4446',
   'ticker': 'TESTING-5',
   'order_id': '0346556f-209a-497b-aaed-921c123cafde',
   'side': 'no',
   'action': '',
   'count': 1,
   'yes_price': 1,
   'no_price': 99,
   'is_taker': False,
   'created_time': '2022-11-28T19:57:54Z'},
  {'trade_id': '707870a0-04b0-4914-90c8-462011dc7627',
   'ticker': 'NGDP-22-C7.5',
   'order_id': '3bdac53a-b774-4f1a-a13c-815483d53f03',
   'side': 'no',
   'action': '',
   'count': 3,
   'yes_price': 23,
   'no_price': 77,
   'is_taker': True,
   'created_time': '2022-11-28T16:01:50Z'},
  {'trade_id': 'ffc9c05e-6c56-418d-ab67-522004889896',
   'ticker': 'NGDP-22-C7.5',
   'order_id': '4408da70-d13c-49b3-9a86-0deb9c90b9dc',
   'side': 'no',
   'action': '',
   'count': 19,
   'yes_price': 23,
   'no_price': 77,
   'is_taker': True,
   'created_time': '2022-11-28T16:00:45Z'},
  {'trade_id': 'ba0a3bee-8fad-4c2a-8792-6842599e2437',
   'ticker': 'TESTING-5',
   'order_id': 'ad1459df-0f72-4c09-8

In [11]:
# you may even want to check on some of your recent positions settled...

settlement_params = {'limit': None,
                        'cursor': None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

{'settlements': []}

In [12]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi. 
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = 'TESTING-5'

order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':'limit',
                    'action':'buy',
                    'side':'no',
                    'count':10,
                    'yes_price':None, # yes_price = 100 - no_price
                    'no_price':30, # no_price = 100 - yes_price
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}

exchange_client.create_order(**order_params)

# EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'limit',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':10,
#                     'yes_price':None, # yes_price = 100 - no_price
#                     'no_price':30, # no_price = 100 - yes_price
#                     'expiration_ts':None,
#                     'sell_position_floor':None,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


# # Market sell order for 12 units Yes on GDPW-22-A3, without flipping position

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)

# # EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'buy',
#                     'side':'no',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)             


{'ticker': 'TESTING-5', 'client_order_id': 'b5b2d443-3707-4215-ab2f-3104b3bf34c6', 'side': 'no', 'action': 'buy', 'count': 10, 'type': 'limit', 'no_price': 30}


{'order': {'order_id': '135403d3-ada5-4fcf-994c-eb433ad33c65',
  'user_id': 'e93aedcf-e3fa-4a76-95c0-24c48b74746d',
  'ticker': 'TESTING-5',
  'status': 'resting',
  'yes_price': 30,
  'no_price': 70,
  'created_time': '2022-11-29T16:44:58Z',
  'taker_fill_count': 0,
  'taker_fill_cost': 0,
  'place_count': 10,
  'decrease_count': 0,
  'maker_fill_count': 0,
  'fcc_cancel_count': 0,
  'close_cancel_count': 0,
  'remaining_count': 10,
  'queue_position': 0,
  'expiration_time': None,
  'taker_fees': 0,
  'action': 'buy',
  'side': 'no',
  'type': 'limit',
  'client_order_id': 'b5b2d443-3707-4215-ab2f-3104b3bf34c6'}}